In [13]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


#!/usr/bin/env python
# coding: utf-8

# In[114]:


import os
import struct
import numpy as np
import torch
import torchvision
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable



import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter


from six.moves import xrange

import umap
import glob
import torch
import cv2
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid


import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"


# In[115]:


#Extracting the palette

#Reading Paintings
path = '/hpcfs/users/a1784072/a1784072/Paintings/*.jpg'
filenames = [img for img in glob.glob(path)]
filenames.sort()
#Reading Generated Paintings
path_1 = '/hpcfs/users/a1784072/a1784072/Generated/*.png'
filenames_1 = [img for img in glob.glob(path_1)]
filenames_1.sort()
# Monet Palette
mp_palette = []
for name in filenames:
    img = cv2.imread(name,cv2.IMREAD_UNCHANGED)
    data = img.reshape((-1, 3))
    data = np.float32(data)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS
    compactness,labels,centers = cv2.kmeans(data,1,None,criteria,10,flags)

    #print('Monet Palette Dominant color is: BGR ({})'.format(centers[0].astype(np.int32)))
    mp_palette.append(centers[0].astype(np.int32))
    
#Generated Monet Palette
gen_palette = []
for name in filenames_1:
    img = cv2.imread(name,cv2.IMREAD_UNCHANGED)
    data = img.reshape((-1, 3))
    data = np.float32(data)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS
    compactness,labels,centers = cv2.kmeans(data,1,None,criteria,10,flags)

    #print('Generated Monet Palette Dominant color is: BGR({})'.format(centers[0].astype(np.int32)))
    gen_palette.append(centers[0].astype(np.int32))
rgb_pal = [i[[2,1,0]] for i in mp_palette]
X = [i/255 for i in rgb_pal]


# In[116]:



trans = transforms.Compose([# remove this
                            transforms.Resize((32, 32)),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
dataset = datasets.ImageFolder('18CDataset', transform=trans)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=128,
    shuffle=True
)


# In[117]:


cuda = True if torch.cuda.is_available() else False


# In[118]:


channels = 3
img_size = 32
bz = 128


# In[119]:


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(512, 1), nn.Sigmoid())

    def forward(self, img):
        #print('img',img.shape)
        out = self.model(img)
        #print("1",out.shape)
        out = out.view(out.shape[0], -1)
        #print("2",out.shape)
        validity = self.adv_layer(out)

        return validity


# In[130]:


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)
        
        
        
        
        
from functools import partial
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)  

def activation_func(activation):
    return  nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['leaky_relu', nn.LeakyReLU(negative_slope=0.01, inplace=True)],
        ['selu', nn.SELU(inplace=True)],
        ['none', nn.Identity()]
    ])[activation]

def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(conv(in_channels, out_channels, *args, **kwargs), nn.InstanceNorm2d(out_channels))

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, activation='relu'):
        super().__init__()
        self.in_channels, self.out_channels, self.activation = in_channels, out_channels, activation
        self.blocks = nn.Identity()
        self.activate = activation_func(activation)
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        x = self.activate(x)
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(
            nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            nn.BatchNorm2d(self.expanded_channels)) if self.should_apply_shortcut else None
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels

class ResNetBasicBlock(ResNetResidualBlock):
    """
    Basic ResNet block composed by two layers of 3x3conv/batchnorm/activation
    """
    expansion = 1
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation_func(self.activation),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),
        )
    
class ResNetLayer(nn.Module):
    """
    A ResNet layer composed by `n` blocks stacked one after the other
    """
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        self.blocks = nn.Sequential(
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion, 
                    out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        )

    def forward(self, x):
        x = self.blocks(x)
        return x
    
    
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.InstanceNorm2d(out_channels),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.InstanceNorm2d(out_channels),
    ) 


In [14]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = img_size // 4
        self.l1 = nn.Sequential(nn.Linear(100, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )
        
        # for conv
        
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.norm1_1 = nn.InstanceNorm2d(64)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # residual blocks
        self.res1 = ResNetLayer(64, 128, block=ResNetBasicBlock, n=1)
        self.res2 = ResNetLayer(128, 256, block=ResNetBasicBlock, n=1)
        self.res3 = ResNetLayer(256, 512, block=ResNetBasicBlock, n=1)
        
        
        
        
        #for deconv
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True) 
        self.dconv_up_4 = double_conv(300 + 512, 256)
        self.dconv_up_3 = double_conv(256 + 256, 128)
        self.dconv_up_2 = double_conv(300 + 128 + 128, 64)
        self.dconv_up_1 = double_conv(300 + 64 + 64, 64)
        self.conv_last = nn.Conv2d(64, 3, 3, padding=1) 

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        x = self.conv_blocks(out)
        
        
        x = F.relu(self.norm1_1(self.conv1_1(x)))
        c4 = self.pool1(x)
        c3 = self.res1(c4)
        c2 = self.res2(c3)
        c1 = self.res3(c2)
        
        
        
        target_palettes_1d = torch.from_numpy(np.array(rgb_pal).flatten()).float().to(device)
        bz, h, w = c1.shape[0], c1.shape[2], c1.shape[3]
        #print('bz', bz)
        #print('\nh',h)
        #print('\nw',w)
        target_palettes = torch.ones(bz, 300, h, w).float().to(device)
        target_palettes = target_palettes.reshape(h, w, bz * 300) * target_palettes_1d.repeat(bz)
        target_palettes = target_palettes.permute(2, 0, 1).reshape(bz, 300, h, w)

        # concatenate target_palettes with c1
        x = torch.cat((c1.float(), target_palettes.float()), 1)
        x = self.dconv_up_4(x)
        x = self.upsample(x)
        #print('C1-X', x.shape)
        
        
        # concatenate c2 with x
        x = torch.cat([c2, x], dim=1)
        #print('After c2-cat',x.shape)
        x = self.dconv_up_3(x)
        x = self.upsample(x)   
        #print('C2-X', x.shape)
        
        # concatenate target_palettes and c3 with x
        bz, h, w = x.shape[0], x.shape[2], x.shape[3]
        target_palettes = torch.ones(bz, 300, h, w).float().to(device)
        target_palettes = target_palettes.reshape(h, w, bz * 300) * target_palettes_1d.repeat(bz)
        target_palettes = target_palettes.permute(2, 0, 1).reshape(bz, 300, h, w)
        #print('Target Pal', target_palettes.shape)
        x = torch.cat([target_palettes.float(), c3, x], dim=1)      
        x = self.dconv_up_2(x)
        x = self.upsample(x)  

        # concatenate target_palettes and c4 with x
        bz, h, w = x.shape[0], x.shape[2], x.shape[3]
        target_palettes = torch.ones(bz, 300, h, w).float().to(device)
        target_palettes = target_palettes.reshape(h, w, bz * 300) * target_palettes_1d.repeat(bz)
        target_palettes = target_palettes.permute(2, 0, 1).reshape(bz, 300, h, w)
        x = torch.cat([target_palettes.float(), c4, x], dim=1) 
        x = self.dconv_up_1(x)
        x = self.upsample(x) 
        #print('X shape (Before:)',x.shape)
        img = self.conv_last(x)
        #print('X shape (After:)', x.shape)
        return img


# In[131]:


# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)


# In[132]:


optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5,0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5,0.999))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


# In[133]:


n_epochs = 20
#latent_dim = 100


# In[ ]:



# In[12]:




In [ ]:
for epoch in range(n_epochs):
    for i, (imgs,_) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 100))))

        # Generate a batch of images
        gen_imgs = generator(z)
        

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
    save_image(gen_imgs.data[:5], "images-203/%d.png" % epoch, nrow=5, normalize=True)
    #save_image(gen_imgs.data[:25], "images-193/%d.png" % epoch, nrow=5, normalize=True)
    #torch.save(generator.state_dict(), "saved_models-203/Gen_", epoch,".pth") 
    #torch.save(discriminator.state_dict(), "saved_models-19/Dis%d.pth" % epoch)          
    torch.save(generator.state_dict(), "saved_models-203/Gen_%d.pth" % (epoch))
    torch.save(discriminator.state_dict(), "saved_models-203/Dis_%d.pth" % (epoch))
# In[ ]:

[Epoch 0/20] [Batch 0/469] [D loss: 0.692815] [G loss: 0.694017]
[Epoch 0/20] [Batch 1/469] [D loss: 0.692451] [G loss: 0.694423]
[Epoch 0/20] [Batch 2/469] [D loss: 0.692833] [G loss: 0.694047]
[Epoch 0/20] [Batch 3/469] [D loss: 0.692588] [G loss: 0.693422]
[Epoch 0/20] [Batch 4/469] [D loss: 0.692881] [G loss: 0.694194]
[Epoch 0/20] [Batch 5/469] [D loss: 0.692834] [G loss: 0.693833]
[Epoch 0/20] [Batch 6/469] [D loss: 0.692826] [G loss: 0.693804]
[Epoch 0/20] [Batch 7/469] [D loss: 0.692866] [G loss: 0.693696]
[Epoch 0/20] [Batch 8/469] [D loss: 0.693077] [G loss: 0.693435]
[Epoch 0/20] [Batch 9/469] [D loss: 0.693101] [G loss: 0.693619]
[Epoch 0/20] [Batch 10/469] [D loss: 0.693060] [G loss: 0.693419]
[Epoch 0/20] [Batch 11/469] [D loss: 0.693073] [G loss: 0.693523]
[Epoch 0/20] [Batch 12/469] [D loss: 0.693132] [G loss: 0.692500]
[Epoch 0/20] [Batch 13/469] [D loss: 0.693619] [G loss: 0.693085]
[Epoch 0/20] [Batch 14/469] [D loss: 0.693342] [G loss: 0.692891]
[Epoch 0/20] [Batch 